In [ ]:
import torch
print(torch.cuda.is_available())  # Should print Ture

True


In [ ]:
from google.colab import files

In [ ]:
!pip install transformers torch datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import json
import random
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset

# Load intents
with open('intents.json', 'r') as f:
    intents = json.load(f)

# Prepare training data
def prepare_training_data(intents):
    training_data = []
    for intent in intents['intents']:
        for pattern in intent['patterns']:
            for response in intent['responses']:
                # Combine pattern and response into a single sequence
                text = f"User: {pattern} Bot: {response}"
                training_data.append(text)
    return training_data

# Fine-tune GPT-2
def fine_tune_gpt2():
    # Load tokenizer and model
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    model = GPT2LMHeadModel.from_pretrained("gpt2")

    # Set padding token to EOS token
    tokenizer.pad_token = tokenizer.eos_token
    print(f"Padding token set to: {tokenizer.pad_token}")

    # Prepare data
    training_data = prepare_training_data(intents)
    print(f"Prepared {len(training_data)} training examples.")

    # Tokenize data and include labels
    tokenized_data = [tokenizer(text, truncation=True, padding="max_length", max_length=128, return_tensors="pt")
                      for text in training_data]
    dataset_dict = {
        "input_ids": [t["input_ids"].squeeze() for t in tokenized_data],
        "attention_mask": [t["attention_mask"].squeeze() for t in tokenized_data],
        "labels": [t["input_ids"].squeeze().clone() for t in tokenized_data]  # Labels are the same as input_ids
    }
    dataset = Dataset.from_dict(dataset_dict)

    # Training arguments
    training_args = TrainingArguments(
        output_dir="./gpt2-zakat",
        num_train_epochs=40,
        per_device_train_batch_size=2,
        save_steps=500,
        save_total_limit=2,
        logging_dir='./logs',
        logging_steps=10,
        report_to="none"  # Disable W&B
    )

    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
    )

    # Train
    trainer.train()

    # Save model and tokenizer
    model.save_pretrained("./gpt2-zakat")
    tokenizer.save_pretrained("./gpt2-zakat")
    print("Model fine-tuned and saved to './gpt2-zakat'")

if __name__ == "__main__":
    fine_tune_gpt2()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Padding token set to: <|endoftext|>
Prepared 88 training examples.


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,4.531600
20,2.730100
30,2.499500
40,1.863300
50,1.630800
60,1.153100
70,1.552600
80,1.253800
90,1.125100
100,0.901600


Model fine-tuned and saved to './gpt2-zakat'


In [ ]:
!zip -r gpt2-zakat.zip ./gpt2-zakat
files.download('gpt2-zakat.zip')

  adding: gpt2-zakat/ (stored 0%)
  adding: gpt2-zakat/special_tokens_map.json (deflated 74%)
  adding: gpt2-zakat/checkpoint-4000/ (stored 0%)
  adding: gpt2-zakat/checkpoint-4000/training_args.bin (deflated 52%)
  adding: gpt2-zakat/checkpoint-4000/trainer_state.json (deflated 85%)
  adding: gpt2-zakat/checkpoint-4000/generation_config.json (deflated 24%)
  adding: gpt2-zakat/checkpoint-4000/rng_state.pth (deflated 25%)
  adding: gpt2-zakat/checkpoint-4000/scheduler.pt (deflated 55%)
  adding: gpt2-zakat/checkpoint-4000/optimizer.pt (deflated 8%)
  adding: gpt2-zakat/checkpoint-4000/config.json (deflated 51%)
  adding: gpt2-zakat/checkpoint-4000/model.safetensors (deflated 7%)
  adding: gpt2-zakat/vocab.json (deflated 68%)
  adding: gpt2-zakat/merges.txt (deflated 53%)
  adding: gpt2-zakat/generation_config.json (deflated 24%)
  adding: gpt2-zakat/tokenizer_config.json (deflated 56%)
  adding: gpt2-zakat/checkpoint-4400/ (stored 0%)
  adding: gpt2-zakat/checkpoint-4400/training_args.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>